In [1]:
import pandas as pd
import numpy as np

# Load and transform the data
data = pd.read_csv('./drive/MyDrive/VagusNerveResearchProject/vns_dataset_threshold_type.csv')

# data = pd.get_dummies(data, columns=['fibre_type'])

data.loc[data['fibre_type'] == 'AB', 'fibre_type'] = 0.0
data.loc[data['fibre_type'] == 'C', 'fibre_type'] = 1.0

data = data.astype('float32')
data.head()

data_ab = data.loc[data['fibre_type'] == 0.0, :]
data_c = data.loc[data['fibre_type'] == 1.0, :]

In [2]:
from sklearn.preprocessing import StandardScaler

# One-Hot Encoding for Categorical Variable
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels]

X_cat = one_hot_encode(data[['fibre_type']].astype('int'), num_classes=2)
X_cat = X_cat.reshape(-1, 2)

X = data.loc[:, ['nerve_a', 'nerve_b', 'activation_level', 'frequency']]
X.loc[:, 'fibre_type_AB'] = X_cat[:, 0]
X.loc[:, 'fibre_type_C'] = X_cat[:, 1]

y = data.loc[:, ['pulse_width', 'amplitude']].values

In [3]:
X_ab = X[X['fibre_type_AB'] == 1.0]
X_c = X[X['fibre_type_C'] == 1.0]
y_ab = y[X['fibre_type_AB'] == 1.0]
y_c = y[X['fibre_type_C'] == 1.0]

# Normalize the continuous features
X_scaler_ab = StandardScaler()
X_ab_norm = X_scaler_ab.fit_transform(X_ab[['nerve_a', 'nerve_b', 'activation_level', 'frequency']])
X_scaler_c = StandardScaler()
X_c_norm = X_scaler_c.fit_transform(X_c[['nerve_a', 'nerve_b', 'activation_level', 'frequency']])

y_scaler_ab = StandardScaler()
y_ab_norm = y_scaler_ab.fit_transform(y_ab)
y_scaler_c = StandardScaler()
y_c_norm = y_scaler_c.fit_transform(y_c)

X_ab_norm = np.hstack([X_ab_norm, X_ab[['fibre_type_AB', 'fibre_type_C']].values])
X_c_norm = np.hstack([X_c_norm, X_c[['fibre_type_AB', 'fibre_type_C']].values])

X_norm = np.vstack([X_ab_norm, X_c_norm])
y_norm = np.vstack([y_ab_norm, y_c_norm])

In [9]:
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch

X_norm = torch.tensor(X_norm, dtype=torch.float32)
y_norm = torch.tensor(y_norm, dtype=torch.float32)

dataset = TensorDataset(X_norm, y_norm)

# Random split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class MDN(nn.Module):
    def __init__(self, input_dim, output_dim, num_mixtures):
        super(MDN, self).__init__()
        self.num_mixtures = num_mixtures
        self.output_dim = output_dim

        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, num_mixtures * (2 * output_dim + 1))

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        params = self.fc3(x)

        means = params[:, :self.num_mixtures * self.output_dim]
        variances = params[:, self.num_mixtures * self.output_dim:2 * self.num_mixtures * self.output_dim]
        weights = params[:, 2 * self.num_mixtures * self.output_dim:]

        means = means.view(-1, self.num_mixtures, self.output_dim)
        variances = torch.exp(variances.view(-1, self.num_mixtures, self.output_dim))
        weights = torch.softmax(weights, dim=1)

        return means, variances, weights

def mdn_loss(y_true, means, variances, weights):
    y_true = y_true.unsqueeze(1).expand_as(means)
    diff = y_true - means
    exponent = -0.5 * torch.sum((diff ** 2) / variances, dim=2)
    normalizer = -0.5 * y_true.size(2) * torch.log(2 * torch.pi * variances).sum(dim=2)
    log_probs = exponent + normalizer
    weighted_log_probs = log_probs + torch.log(weights)
    log_sum_exp = torch.logsumexp(weighted_log_probs, dim=1)
    return -log_sum_exp.mean()

In [5]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [7]:
import optuna


def create_model(trial):
    input_dim = 6
    output_dim = 2
    num_mixtures = trial.suggest_int('num_mixtures', 3, 10)
    num_units = trial.suggest_int('num_units', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    model = MDN(input_dim, output_dim, num_mixtures)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    return model, optimizer

def objective(trial):
    model, optimizer = create_model(trial)
    criterion = mdn_loss
    num_epochs = 50

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            means, variances, weights = model(batch_x)
            loss = criterion(batch_y, means, variances, weights)
            loss.backward()
            optimizer.step()

    # Validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            means, variances, weights = model(batch_x)
            loss = criterion(batch_y, means, variances, weights)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    return -val_loss  # Negate the loss to ensure Optuna maximizes the log-likelihood

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)


best_params = study.best_params
print('Best parameters: ', best_params)


[I 2024-08-09 14:49:01,430] A new study created in memory with name: no-name-3c298b0d-7e62-49cd-bfc0-31de1a18a657
<ipython-input-6-74e646f4418b>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[W 2024-08-09 14:49:02,888] Trial 0 failed with parameters: {'num_mixtures': 8, 'num_units': 53, 'learning_rate': 0.00036072786090956367} because of the following error: NameError("name 'train_loader' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-6-74e646f4418b>", line 24, in objective
    for batch_x, batch_y in train_loader:
NameError: name 'train_loader' is not defined
[W 2024-08-09 14:49:02,890] Trial 0 

NameError: name 'train_loader' is not defined

In [10]:
# Create the best model with optimal hyperparameters

best_params = {'num_mixtures': 3, 'num_units': 37, 'learning_rate': 0.00013292213551810696}

best_model, best_optimizer = create_model(optuna.trial.FixedTrial(best_params))

# Training the best model
num_epochs = 50
criterion = mdn_loss

for epoch in range(num_epochs):
    best_model.train()
    for batch_x, batch_y in train_loader:
        best_optimizer.zero_grad()
        means, variances, weights = best_model(batch_x)
        loss = criterion(batch_y, means, variances, weights)
        loss.backward()
        best_optimizer.step()

# Evaluate the best model
best_model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_x, batch_y in val_loader:
        means, variances, weights = best_model(batch_x)
        loss = criterion(batch_y, means, variances, weights)
        test_loss += loss.item()

test_loss /= len(val_loader)
print('Test loss: ', test_loss)


<ipython-input-7-6fd9041719f9>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


Test loss:  -1.4457371273064261
